In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
df_devs = pd.read_csv('clean_datasets/steam_developers.csv') #done
df_pubs = pd.read_csv('clean_datasets/steam_publishers.csv') #done
df_category = pd.read_csv('clean_datasets/steam_categories.csv') #done
df_steam_descriptions = pd.read_csv('clean_datasets/steam_description_data.csv') #done
df_steam = pd.read_csv('clean_datasets/steam.csv') #done
df_genres = pd.read_csv('clean_datasets/steam_genres.csv') #done
df_steam_requirements = pd.read_csv('clean_datasets/steam_requirements_data.csv') #done
df_steam_tags = pd.read_csv('clean_datasets/steamspy_tag_data.csv') #done
df_steam_achievements = pd.read_csv('clean_datasets/steam_achievements.csv') #done
df_steam_support = pd.read_csv('clean_datasets/steam_support.csv') #done

In [3]:
def group_into_lists(df, group, column):
    return df.groupby(group)[column].apply(list)

In [4]:
df_steam_json = df_steam

# group devs and publishers into lists
df_devs_lists = group_into_lists(df_devs, "appid", "developer")
df_steam_json = pd.merge(df_steam_json, df_devs_lists, on="appid", how="left")

df_pubs_lists = group_into_lists(df_pubs, "appid", "publisher")
df_steam_json = pd.merge(df_steam_json, df_pubs_lists, on="appid", how="left")

df_steam_json = df_steam_json.rename(columns={"developer":"developers", "publisher":"publishers"})

In [5]:
# group genres and categories into lists
df_steam_json['genres'] = df_genres.apply(lambda x: list(x.index[x == True]), axis=1)
df_steam_json['categories'] = df_category.apply(lambda x: list(x.index[x == True]), axis=1)

In [6]:
# join descriptions
df_steam_descriptions = df_steam_descriptions.rename(columns={"steam_appid":"appid"})
df_steam_json = pd.merge(df_steam_json, df_steam_descriptions, on="appid", how="left")

In [7]:
# join requirements
df_steam_requirements = df_steam_requirements.rename(columns={"steam_appid":"appid"})
df_steam_json = df_steam_json.merge(df_steam_requirements, on="appid", how="left")

In [8]:
# join support
df_steam_support = df_steam_support.rename(columns={"steam_appid":"appid"})
df_steam_json = df_steam_json.merge(df_steam_support, on="appid", how="left")

In [9]:
# Writing data to json file

f = open("solr_datasets/steam.json", "w")

f.write("[")
for row in df_steam_json.iterrows():
    row[1].to_json(f)
    if not row[0] == len(df_steam_json) - 1:
        f.write(",")
    f.write("\n")

f.write("]")
f.close()

In [10]:
f = open("solr_datasets/steam.json")
steam_json = json.load(f)
f.close()

In [11]:
for game in steam_json:
    pd_game_tag = df_steam_tags[df_steam_tags["appid"] == game["appid"]]
    pd_game_tag = pd_game_tag.drop(columns=pd_game_tag.columns[(pd_game_tag==0).any()])
    game["_tags_"] = json.loads(pd_game_tag.iloc[0].to_json())



In [13]:
for game in steam_json:
    achievements = []
    pd_game_achievements = df_steam_achievements[df_steam_achievements["appid"] == game["appid"]]
    pd_game_achievements = pd_game_achievements.drop(columns=['appid'])

    for row in pd_game_achievements.iterrows():
        achievements.append(json.loads(row[1].to_json()))

    if not pd_game_achievements.empty:
        game["achievements"] = achievements
    else:
        game["achievements"] = []

In [14]:
print(steam_json[0])

{'appid': 10, 'name': 'Counter-Strike', 'release_date': '01/11/2000', 'english': True, 'required_age': 0, 'positive_ratings': 124534, 'negative_ratings': 3339, 'average_playtime': 17612, 'median_playtime': 317, 'price': 7.19, 'owners_range': '[10000000, 20000000]', 'windows': True, 'linux': True, 'mac': True, 'developers': ['Valve'], 'publishers': ['Valve'], 'genres': ['Action'], 'categories': ['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Valve Anti-Cheat enabled'], 'detailed_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.", 'about_the_game': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. All

In [16]:
f1 = open("solr_datasets/steam1.json", "w")
f2 = open("solr_datasets/steam2.json", "w")

for game in steam_json[:int(len(steam_json)/2)]:
    f1.write(json.dumps(game) + "\n")

for game in steam_json[int(len(steam_json)/2):]:
    f2.write(json.dumps(game) + "\n")

# json.dump(steam_json[:int(len(steam_json)/2)],f1)
# json.dump(steam_json[int(len(steam_json)/2):],f2)

f1.close()
f2.close()

In [27]:
f3 = open("elastic_datasets/steam1.json", "w")
f4 = open("elastic_datasets/steam2.json", "w")

i = 1
for doc in steam_json[:int(len(steam_json)/2)]:
    f3.write("{ \"index\" : { \"_index\" : \"games_test\", \"_id\" : \"" + str(i) + "\" } }" + "\n")
    f3.write(json.dumps(doc) + "\n")
    i = i + 1

for doc in steam_json[int(len(steam_json)/2):]:
    f4.write("{ \"index\" : { \"_index\" : \"games_test\", \"_id\" : \"" + str(i) + "\" } }")
    f4.write(json.dumps(doc) + "\n")
    i = i + 1




In [21]:
import json
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
i = 0
#with open('solr_datasets/steam1_copy.json') as steam_json_copy:

for json_doc in steam_json:
        i = i + 1
        es.index(index='games', doc_type='games', id=i, body=json.dumps(json_doc))

